### Titanic

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = sns.load_dataset('titanic')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [5]:
data.describe() # 기술 통계량 제공

,survived,pclass,age,sibsp,parch,fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


age, embarked, deck, embarked_town에 결측치가 존재한다는 것을 알 수 있다.

#### Data Preprocessing

In [6]:
data['age'].fillna(data['age'].mean(), inplace = True)

In [7]:
data['embarked'].fillna(data['embarked'].mode()[0], inplace = True)

Text데이터로 되어있는 것들은 `LabelEncoder`를 이용해 인코딩을 해 준다

In [8]:
from sklearn.preprocessing import LabelEncoder

data['sex'] = LabelEncoder().fit_transform(data['sex'])

In [9]:
data['embarked'] = LabelEncoder().fit_transform(data['embarked'])

In [10]:
data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,1,22.000000,1,0,7.2500,2,Third,man,True,NaN,Southampton,no,False
1,1,1,0,38.000000,1,0,71.2833,0,First,woman,False,C,Cherbourg,yes,False
2,1,3,0,26.000000,0,0,7.9250,2,Third,woman,False,NaN,Southampton,yes,True
3,1,1,0,35.000000,1,0,53.1000,2,First,woman,False,C,Southampton,yes,False
4,0,3,1,35.000000,0,0,8.0500,2,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.000000,0,0,13.0000,2,Second,man,True,NaN,Southampton,no,True
887,1,1,0,19.000000,0,0,30.0000,2,First,woman,False,B,Southampton,yes,True
888,0,3,0,29.699118,1,2,23.4500,2,Third,woman,False,NaN,Southampton,no,False
889,1,1,1,26.000000,0,0,30.0000,0,First,man,True,C,Cherbourg,yes,True
